# LangChain QuickStart Tutorial

I'm essentially following this QuickStart:

https://python.langchain.com/docs/get_started/quickstart

However, I've modified some of the statements and queries. I'm also using Azure OpenAI so some of the imports differ.

In [7]:
import dotenv
import os

# Use a local .env file to populate environment variables
dotenv.load_dotenv(override=True)

model_deployment = os.getenv('MODEL_DEPLOYMENT')
embedding_deployment = os.getenv('EMBEDDING_DEPLOYMENT')


This first cell creates a basic chain with a system prompt, the llm, and an output parser.
It then asks the LLM "Who is Martin Peck?" and prints the response.
It should give a response similar to the following:

> I'm sorry, but as a software engineer, I don't have specific information on individuals unless they are widely recognized in the field of software engineering or technology. Could you please provide more context?


In [8]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = AzureChatOpenAI(deployment_name=model_deployment)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a software engineer. When responding, be brief and to the point."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "Who is Martin Peck?"})


"I'm sorry, but as a software engineer, I don't have specific information on individuals unless they are widely recognized in the field of technology or software engineering. Martin Peck doesn't appear to be a notable figure in these areas based on the information available to me."

This next cell uses the WebBaseLoader (which, under the covers, uses BeautifulSoup) to scrape my blog "About" page

In [9]:
from langchain_community.document_loaders import WebBaseLoader

# scrape the web page
loader = WebBaseLoader("https://martinpeck.com/about/")
docs = loader.load()

Next, we prepare an embedding model (again, using the Azure OpenAI service), chunk the scraped web page, and then generate and store embeddings in a local FAISS vector store

In [10]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# prep the embedding model
embeddings = AzureOpenAIEmbeddings(azure_deployment=embedding_deployment)

# split the scraped web page into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# create embeddings and store them in a local FAISS vector store
vector = FAISS.from_documents(documents, embeddings)

Finally, we set up a chat template that allows us to inject both context and a question into the chat.
When we invoke this chain the LLM is given the context from the vector store, and this time when asked "Who is Martin Peck" should respond with something similar to the following:

> Martin Peck is a software engineer with over 20 years of experience in writing software and leading software development teams. He is currently a Software Engineering Manager at Microsoft, where he manages a team of Software Engineers. He has also worked for blinkbox and was previously the Director of Engineering for a not-for-profit called Code Club. In addition to his work in software, he also draws and publishes cartoons, some of which can be found for sale at Redbubble and Zazzle.

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "Who is Martin Peck?"})
print(response["answer"])

Martin Peck is a software developer and team leader with over 20 years of experience. He has worked for Microsoft, where he is currently a Software Engineering Manager, and for a not-for-profit organization called Code Club, where he was the Director of Engineering. He also worked for blinkbox in various roles. In addition to his professional roles, Martin Peck is a volunteer who runs a coding club at a local school. He also draws and publishes cartoons.
